# Import Libraries

In [ ]:
import os
import sys
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt

GFE_DIR = ''
sys.path.append(GFE_DIR)
import JKsKIRT as gskirt

from rur import uri, uhmi

In [ ]:
SioverSil = 0.163 
# Silicate in NC --- assuming a fixed amorphous olivine composition(MgFeSiO4).
#                    assuming pyroxene with MgFeSi2O6 instead of olivine 
#                    typically increase the amount of dust mass released as silicates by 20%.

# Test

## load sim raw data

In [ ]:
nout = 341

repo = '/storage7/NewCluster2/'
path_in_repo = 'snapshots'
mode = 'nc'

boxrad = 15*np.sqrt(3)

In [ ]:
snap = uri.RamsesSnapshot(repo=repo, 
                          path_in_repo=path_in_repo, 
                          iout=nout, 
                          mode=mode, 
                          longint=False)

#boxtokpc = 1/snap.unit['kpc']#/snap.params['aexp']

In [ ]:
gals_tot, pid = uhmi.HaloMaker.load( snap, 
                                    galaxy=True,
                                    load_parts=True,
                                    double_precision=True)
gals_tot.sort(order='m')

In [ ]:
mlow = 10
mupp = 13

gals = gals_tot[(10**mlow<=gals_tot['mvir'])*(gals_tot['mvir']<10**mupp)]
print(gals['id'])

In [ ]:
idgal = 518

In [ ]:
gal = gals[gals['id']==idgal]

In [ ]:
snap.clear()
snap = uri.RamsesSnapshot(repo=repo, 
                          path_in_repo=path_in_repo, 
                          iout=nout, 
                          mode=mode)

boxtokpc = 1/snap.unit['kpc']
snap.set_box_halo(gal, 
                  use_halo_radius=False, 
                  radius=boxrad/boxtokpc)

star = snap.get_part(pname='star')

snap.clear()
snap = uri.RamsesSnapshot(repo=repo, 
                          path_in_repo=path_in_repo, 
                          iout=nout, 
                          mode=mode)
snap.set_box_halo(gal, 
                  use_halo_radius=False, 
                  radius=boxrad/boxtokpc)
snap.get_cell()
cell = snap.cell    

In [ ]:
mC_small = cell['d1']*cell['m','Msun']
mC_large = cell['d2']*cell['m','Msun']
mSil_small = cell['d3']*cell['m','Msun']/SioverSil
mSil_large = cell['d4']*cell['m','Msun']/SioverSil

In [ ]:
print('Total small Graphite Mass = %s (log Msun)'%(np.round(np.log10(np.sum(mC_small)),2)))
print('Total large Graphite Mass = %s (log Msun)'%(np.round(np.log10(np.sum(mC_large)),2)))
print('Total small Silicate Mass = %s (log Msun)'%(np.round(np.log10(np.sum(mSil_small)),2)))
print('Total large Silicate Mass = %s (log Msun)'%(np.round(np.log10(np.sum(mSil_large)),2)))

## load param file

In [ ]:
repo_par = '/home/jangjk/Project/JKsKIRT/example/'
fname_par = 'parameter_setting'

try:
    par = reload(sys.modules[fname_par])
except:
    pass

sys.path.append(repo_par)
par = __import__(fname_par)

In [ ]:
par.skirt_dir = '/storage5/scratch/library/SKIRT/release/SKIRT/main/'
par.N_phot = 3e7

par.repo = f'/storage8/jangjk/mock/NC/gal_{nout:04d}/{idgal:04d}/'

In [ ]:
print('Total photon packet Number = 10^{%s}'%(np.round(np.log10(par.N_phot),2)),end='\n\n')
print('Redshift of the target = %s'%(par.z_red))
print('Distance to the target = %s (>0 only when z_red = 0)'%(par.inst_dist),end='\n\n')
print('Field-of-View (X-direction) = %s %s'%(par.fov_X,par.fov_unit))
print('Field-of-View (Y-direction) = %s %s'%(par.fov_Y,par.fov_unit),end='\n\n')
print('Using on-the-fly-dust info = %s'%(par.on_the_fly_dust),end='\n\n')
print('The file will be saved in %s/%s'%(par.repo,par.repo_output),end='\n\n')

## execute SKIRT 

In [ ]:
gskirt.utils.execute_SKIRT.make_INSKI(
                                   boxtokpc=boxtokpc,
                                   x_s=star['x'],y_s=star['y'],z_s=star['z'],
                                   vx_s=star['vx','km/s'],vy_s=star['vy','km/s'],vz_s=star['vz','km/s'],
                                   m_s=star['m','Msun'],m0_s=star['m0','Msun'],
                                   age_s=star['age','Gyr'],metal_s=star['metal'],
                                   x_c=cell['x'],y_c=cell['y'],z_c=cell['z'],lvl_c=cell['level'],
                                   vx_c=cell['vx','km/s'],vy_c=cell['vy','km/s'],vz_c=cell['vz','km/s'],
                                   m_c=cell['m','Msun'],
                                   T_c=cell['T','K'],metal_c=cell['metal'],
                                   param=par,
                                   pos_ctr=[gal['x'],gal['y'],gal['z']],
                                   vel_ctr=[gal['vx'],gal['vy'],gal['vz']],
                                   mC_small=mC_small,
                                   mC_large=mC_large,
                                   mSil_small=mSil_small,
                                   mSil_large=mSil_large,
                                  )
